# Trabalho - INF325 2020

## Equipe 8:

 - Aline Souza Silva (RG: 36.783.233-1)
 - Erik Borges (RG: 15.434.065)
 - Lucas Trinquinato (RG: 50.260.503-0)
 - Paulo Sérgio Marchioreto (27.303.073-5)
 - Renato César Alves de Oliveira (RG: 36.536.287-6)


### Instalando módulo de conexão com o Mongo

In [ ]:
!pip install pymongo

### Instalando módulo para impressão de resultados

In [ ]:
!pip install pprintpp

### Importando módulo de conexão com o Mongo

In [39]:
from pymongo import MongoClient

### Importando módulo para impressão de resultados

In [28]:
from pprintpp import pprint

### Conectando na base de dados Mongo

In [29]:
mongoclient = MongoClient("mongo", 27017)

### Selecionando base de dados

In [30]:
db = mongoclient.marketplace_engcomp_unicamp

### Lojistas com maior número de produtos disponíveis na plataforma

In [43]:
result = db.products.aggregate(
    [
        {
            "$match": {
                "status": "Actived"
            }
        },
        {
            "$group": {
                "_id": "$seller",
                "totalItems": {
                    "$sum": 1
                }
            }
        },
        {
            "$sort": {
                "totalItems": -1
            }
        },
        {
            "$limit": 5
        }
    ]
)

pprint(list(result))

[
    {
        '_id': {
            'cpfCnpj': '12345678900',
            'description': 'Melhor vendedor do Brasil',
            'name': 'Vendedor 1',
            'photo': 'https://image.flaticon.com/icons/svg/2039/2039075.svg',
            'rating': 4,
        },
        'totalItems': 2,
    },
    {
        '_id': {
            'cpfCnpj': '12345678901',
            'description': 'Melhor vendedor do Brasil',
            'name': 'Vendedor 1',
            'photo': 'https://image.flaticon.com/icons/svg/2039/2039075.svg',
            'rating': 4,
        },
        'totalItems': 1,
    },
]


### Lojistas com maior número de produtos disponíveis na plataforma (fazendo join com a tabela sellers para trazer todos os dados do lojista)

In [48]:
result = db.products.aggregate(
    [
        {
            "$match": {
                "status": "Actived"
            }
        },
        {
            "$group": {
                "_id": "$seller.cpfCnpj",
                "totalItems": {
                    "$sum": 1
                }
            }
        },
        {
            "$sort": {
                "totalItems": -1
            }
        },
        {
            "$limit": 5
        },
        {
            "$lookup": {
                "from": "sellers",
                "localField": "_id",
                "foreignField": "cpfCnpj",
                "as": "seller"
            }
        }
    ]
)

pprint(list(result))

[
    {
        '_id': '12345678900',
        'seller': [
            {
                '_id': ObjectId('5f5ae16aa8359483f6fd4fcc'),
                'address': {
                    'city': 'Belo Horizonte',
                    'country': 'Brasil',
                    'district': 'Bairro Santo Antonio',
                    'number': 123,
                    'state': 'Minas Gerais',
                    'street': 'Rua das Sucesso',
                    'zipcode': '30360-250',
                },
                'categories': ['Eletrodomesticos', 'Moveis', 'Variedades'],
                'cpfCnpj': '12345678900',
                'createdAt': datetime.datetime(2020, 9, 10, 8, 15, 39, 736000),
                'description': 'Melhor vendedor do Brasil.',
                'name': 'Vendedor 1',
                'photo': 'https://image.flaticon.com/icons/svg/2039/2039075.svg',
                'rating': 4,
                'status': 'Actived',
                'type': 'PF',
                'updatedAt':

### 5 produtos mais comuns no marketplace

In [45]:
result = db.products.aggregate([
    {
        "$group": {
            "_id": "$productType",
            "total": {
                    "$sum": 1
                }
        }
    },
    {
        "$sort": {
            "total": -1
        }
    },
    {
        "$limit": 5
    }
])

pprint(list(result))

[{'_id': 'smartphone', 'total': 3}]


### 5 produtos mais comuns entre diferentes lojistas (produtos que aparecem em mais lojistas diferentes)

In [46]:
result = db.products.aggregate([
    {
        "$group": {
            "_id": "$productType",
            "sellers": {
                    "$addToSet": "$seller.cpfCnpj"
                }
        }
    },
    {
        "$project": {
            "_id": 1,
            "qtySellers": { "$size": "$sellers" }
        }
    },
    {
        "$sort": {
            "qtySellers": -1
        }
    },
    {
        "$limit": 5
    }
])

pprint(list(result))

[{'_id': 'smartphone', 'qtySellers': 2}]


### 10 produtos mais recomendados de acordo com o "rating" do fornecedor e do produto

In [54]:
result = db.products.find().sort("seller.rating", -1).sort("rating", -1).limit(10)
        
pprint(list(result))

[
    {
        '_id': ObjectId('5f5ad595a8359483f6fd4bcb'),
        'categories': ['Eletronicos'],
        'createdAt': datetime.datetime(2020, 9, 10, 8, 15, 39, 736000),
        'description': 'O melhor smartphone do mercado pelo melhor preço',
        'manufactureDetails': {
            'modelNumber': 'A123X',
            'releaseDate': datetime.datetime(2020, 9, 1, 0, 0),
        },
        'photos': [
            'https://www.digithaigroup.com/wp-engine/wp-content/uploads/2015/09/products.png',
            'https://www.digithaigroup.com/wp-engine/wp-content/uploads/2015/09/products.png',
            'https://www.digithaigroup.com/wp-engine/wp-content/uploads/2015/09/products.png',
        ],
        'pricing': {'price': 1000},
        'productType': 'smartphone',
        'quantity': 99,
        'rating': 5,
        'seller': {
            'cpfCnpj': '12345678900',
            'description': 'Melhor vendedor do Brasil',
            'name': 'Vendedor 1',
            'photo': 'https:

### 5 categorias com mais produtos disponíveis

In [55]:
result = db.products.aggregate(
    [
        {
            "$match": {
                "status": "Actived"
            }
        },
        {
            "$unwind": "$categories"
        },
        {
            "$group": {
                "_id": "$categories",
                "totalItems": {
                    "$sum": 1
                }
            }
        },
        {
            "$sort": {
                "totalItems": -1
            }
        },
        {
            "$limit": 5
        }
    ]
)

pprint(list(result))

[{'_id': 'Eletronicos', 'totalItems': 3}]
